In [ ]:
import pandas as pd
train = pd.read_csv('../input/nyc-taxi-trip-duration/train.zip')
test = pd.read_csv('../input/nyc-taxi-trip-duration/test.zip')
submission = pd.read_csv('../input/nyc-taxi-trip-duration/sample_submission.zip')

In [ ]:
train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'])
train['dropoff_datetime'] = pd.to_datetime(train['dropoff_datetime'])
test['pickup_datetime'] = pd.to_datetime(test['pickup_datetime'])

In [ ]:
train['pickup_hour'] = train['pickup_datetime'].dt.hour
test['pickup_hour'] = test['pickup_datetime'].dt.hour

In [ ]:
columns = ['passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'pickup_hour']
X_train = train[columns]
Y_train = train['trip_duration']
X_test = test[columns]

In [ ]:
X_train.shape, Y_train.shape, X_test.shape

In [ ]:
X_train_norm = (X_train - X_train.mean()) / X_train.std()
Y_train_norm = (Y_train - Y_train.mean()) / Y_train.std()
X_test_norm = (X_test - X_test.mean()) / X_test.std()

In [ ]:
X_train_norm.mean()

In [ ]:
X_train_norm.std()

In [ ]:
Y_train_norm.mean()

In [ ]:
Y_train_norm.std()

In [ ]:
X_test_norm.mean()

In [ ]:
X_test_norm.std()

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential() 
model.add(tf.keras.layers.Dense(128, activation = 'relu', input_shape = (6,) ))
model.add(tf.keras.layers.Dense(128, activation = 'relu'))
model.add(tf.keras.layers.Dense(128, activation = 'relu'))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dense(1))

In [ ]:
model.summary()

# 순전파(Foward Propagation) : 입력층에서 출력층 방향으로 예측값의 연산이 진행되는 과정

In [ ]:
model.compile(loss = tf.keras.losses.mse, #실제값과 예측값의 차이를 수치화하는 함수
             optimizer = tf.keras.optimizers.Adam(lr = 0.0001), #손실 함수의 값을 줄여나가면서 학습하는 방법은 어떤 옵티마이저를 사용하느냐에 따라 달라짐. lr(Learning Rate) 학습율 
             metrics = ['mse'])

# loss
classifier문제는 CrossEntropy, 
regression문제는 MSE(Mean Squared Error)

Binary classifier는 binary_crossentropy 사용,
Multi-Class Classifier는 categorical_crossentropy 사용

# Optimizer
배치 경사 하강법(Batch Gradient Descent) : 오차(loss)를 구할 때 전체 데이터를 고려함. 한 번의 에포크에 모든 매개변수 업데이트를 단 한 번 수행함.

확률적 경사 하강법(Stochastic Gradient Descent, SGD) : 랜덤으로 선택한 하나의 데이터만 고려함. 매개변수 변경폭이 불규칙하지만, 속도는 배치 경사 하강법보다 빠름.

미니 배치 경사 하강법(Mini-Batch Gradient Descent): 정해진 양에 대해서만 계산하여 매개 변수의 값을 조정하는 경사 하강법. 전체 데이터를 계산하는 것보다 빠르고, SGD보다 안정적임.

모멘텀(Momentum) SGD : 모멘텀 SGD는 경사 하강법에 관성을 더 해줌. 기존의 경사 하강법은 로컬 미니멈에 도달하였을 때에 기울기가 0이라서 계산이 끝나지만, 모멘텀 방법은 관성의 힘을 빌려 로컬 미니멈에서 탈출함.

아다그라드(Adagrad): 각 매개변수에 서로 다른 학습률을 적용시킴. 변화가 많은 매개변수는 학습률이 작게 설정되고, 변화가 적은 매개변수는 학습률을 높게 설정시킴.

알엠에스프롭(RMSprop) : 아다그라드는 학습을 계속 진행한 경우, 학습률이 지나치게 떨어진다는 단점이 있음. 이를 다른 수식으로 대체함.

아담(Adam) : 알엠에스프롭과 모멘텀 두 가지를 합친 듯한 방법, 방향과 학습률(스텝사이즈) 두 가지를 모두 잡기 위한 방법

In [ ]:
history = model.fit(X_train_norm, Y_train_norm, batch_size = 300, epochs = 30, validation_split = 0.2) #batch_size=300

In [ ]:
pd.DataFrame(history.history)

In [ ]:
pd.DataFrame(history.history).reset_index()

In [ ]:
pd.DataFrame(history.history).reset_index().plot('index','loss')

In [ ]:
pd.DataFrame(history.history).reset_index().plot('index','val_loss')

In [ ]:
Y_train_norm

In [ ]:
submission['trip_duration'] = model.predict(X_test_norm) * Y_train.std() + Y_train.mean()

In [ ]:
submission['trip_duration']

In [ ]:
submission.to_csv('NYTaxi_NeuralNetwork_batchsize300.csv', index=False)